In [ ]:
import json
import sys
import threading
import websocket
import simplejson
from time import sleep
from datetime import datetime
import logging
import os
from RofexAPI import PJS_RESTv3 as pjs
from RofexAPI import MDmessages as md
sys.path.append('C:/Users/pauli/Documents/python/OptionModelsV3/RofexAPI')


In [2]:
logging.basicConfig()

In [3]:
#test funcion datetime
year = str(datetime.today().year)
day = str(datetime.today().day)
if(len(day)==1):
    day = "0"+day
month = str(datetime.today().month)
if(len(month)==1):
    month = "0"+month
print(year)
print(month)
print(day)


2018
11
14


In [4]:
#listaMensajes = []
#subsMDEvents = []
subsOREvents = []
mensajes=0
MSG_MDSuscription = '{{"type":"smd","level":1,"entries":[{e}],"products":[{s}]}}'
MSG_OSSuscription = '{{"type":"os","account":{{"id":"{a}"}}}}'
#MSG_Symbol = '{{"symbol":"{s}","marketId":"ROFX"}}'
MSG_Double_Quotes = '"{s}"'
#global mensajes

In [5]:
def on_message(ws, message):
           
    try:
        # print ("en el try")
        # Valido Mensaje entrante
        msg = simplejson.loads(message)
        msgType = msg['type'].upper()


        if msgType == 'MD':
            
            global mensajes
            mensajes+=1
            print("Mensajes:---->", mensajes)
            #print("--entra en MD--------------------------------------------")
            md.MDincoming(msg)
            
            
        elif msgType == 'OR':
            print ("En Mensaje OR")
            print(msg)
        else:
            print ("Tipo de Mensaje Recibido No soportado: " + msg)
    except:
        print ("Error al procesar mensaje recibido: " + message)

def on_error(ws, error):
    print ("Salio por error")
    print (error)
    ws.close()
    
def on_close(ws):
    print ("### connection closed ###")
    
def on_open(ws):    
    
    print ("On Conection Open...")
    

In [6]:
def initWS():
    # 1st set urls
    # 2nd user data 
    entorno=1
    pjs.setURLs()
    pjs.init("balanz","balanz","555",entorno)
    
    #parametros contratos a suscribir
    type_="smd"
    level_="1"  #? prof mercado
    marketId_="ROFX"
       
    
    symbols=pjs.suscribirContratos()
    print("Contratos suscriptos:")
    for s in symbols:
        print(s)
    print('\n')
    
    
    isLogin = pjs.login()
    if isLogin:
        
        # control
        print("Conexion establecida...")
        print('Token:'+' '+pjs.token)
        # end control
        
        headers = {'X-Auth-Token:{token}'.format(token=pjs.token)}

        ws = websocket.WebSocketApp(pjs.activeWSEndpoint,
                                         on_message=on_message,
                                         on_error=on_error,
                                         on_close=on_close,
                                         on_open=on_open,
                                         header=headers)

        wst = threading.Thread(target=ws.run_forever, kwargs={"ping_interval" :5})
        
        wst.start()

        # Esperamos a que la conexion ws se establezca
        conn_timeout = 5
        #conn_timeout = 50 #y nada
        sleep(1)


        while not ws.sock.connected and conn_timeout:
            sleep(1)
            conn_timeout -= 1
        else:
            
        
            for sym in symbols:
                
                aaa=pjs.buildMessage(type_,level_,marketId_,sym)
                ws.send(aaa)
                
                print("Sent Suscription msg",aaa)
                           
                print("Receiving...")
                
                sleep(1) #y nada

    else:
        print( "Error al autenticarse...")


if __name__ == "__main__":
    initWS()
    

setURLs Ok...
init() Ok...
Contratos suscriptos:
RFX20Dic18
DoDic18


login() OK...
Conexion establecida...
Token: VX8lqBL3M3IPZlQd7av6/tTxSDlV9i5FPjNAuA7efvE=
On Conection Open...
Sent Suscription msg {"type":"smd","level":1, "entries":["BI", "OF"],"products":[{"symbol":"RFX20Dic18","marketId":"ROFX"}]}
Receiving...
Mensajes:----> 1
En MD incoming ****************
RFX20Dic18 -----> 38465 / 38500  size-->  13 / 5 timestamp : 1542204861506
--------------------------------------------------------------------------------------------
Sent Suscription msg {"type":"smd","level":1, "entries":["BI", "OF"],"products":[{"symbol":"DoDic18","marketId":"ROFX"}]}
Receiving...
Mensajes:----> 2
En MD incoming ****************
DoDic18 -----> 39.1 / 39.11  size-->  260 / 32 timestamp : 1542204862504
--------------------------------------------------------------------------------------------
Mensajes:----> 3
En MD incoming ****************
RFX20Dic18 -----> 38465 / 38500  size-->  17 / 5 timestamp : 1542

Mensajes:----> 37
En MD incoming ****************
RFX20Dic18 -----> 38500 / 38530  size-->  5 / 100 timestamp : 1542204958497
--------------------------------------------------------------------------------------------
Mensajes:----> 38
En MD incoming ****************
RFX20Dic18 -----> 38500 / 38530  size-->  5 / 85 timestamp : 1542204961863
--------------------------------------------------------------------------------------------
Mensajes:----> 39
En MD incoming ****************
RFX20Dic18 -----> 38500 / 38530  size-->  5 / 80 timestamp : 1542204962364
--------------------------------------------------------------------------------------------
Mensajes:----> 40
En MD incoming ****************
RFX20Dic18 -----> 38500 / 38530  size-->  75 / 80 timestamp : 1542204963834
--------------------------------------------------------------------------------------------
Mensajes:----> 41
En MD incoming ****************
RFX20Dic18 -----> 38500 / 38530  size-->  70 / 80 timestamp : 1542204964836


Mensajes:----> 75
En MD incoming ****************
RFX20Dic18 -----> 38530 / 38600  size-->  70 / 25 timestamp : 1542205033762
--------------------------------------------------------------------------------------------
Mensajes:----> 76
En MD incoming ****************
RFX20Dic18 -----> 38530 / 38600  size-->  72 / 26 timestamp : 1542205034263
--------------------------------------------------------------------------------------------
Mensajes:----> 77
En MD incoming ****************
RFX20Dic18 -----> 38530 / 38600  size-->  72 / 27 timestamp : 1542205034763
--------------------------------------------------------------------------------------------
Mensajes:----> 78
En MD incoming ****************
DODic18 -----> 39.0 / 39.085  size-->  5 / 15 timestamp : 1542205035378
--------------------------------------------------------------------------------------------
Mensajes:----> 79
En MD incoming ****************
RFX20Dic18 -----> 38530 / 38600  size-->  42 / 27 timestamp : 1542205036494
--

Mensajes:----> 113
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38529  size-->  16 / 1 timestamp : 1542205096509
--------------------------------------------------------------------------------------------
Mensajes:----> 114
En MD incoming ****************
DODic18 -----> 38.95 / 39.049  size-->  55 / 15 timestamp : 1542205097972
--------------------------------------------------------------------------------------------
Mensajes:----> 115
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38529  size-->  17 / 1 timestamp : 1542205098234
--------------------------------------------------------------------------------------------
Mensajes:----> 116
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38529  size-->  7 / 1 timestamp : 1542205101190
--------------------------------------------------------------------------------------------
BID Size of  RFX20Dic18 -----------------> 7
Mensajes:----> 117
En MD incoming ****************
RFX20Dic18 -----> 38460 / 3852

Mensajes:----> 150
En MD incoming ****************
RFX20Dic18 -----> 38400 / 38490  size-->  15 / 16 timestamp : 1542205192003
--------------------------------------------------------------------------------------------
Mensajes:----> 151
En MD incoming ****************
DODic18 -----> 38.9 / 39.0  size-->  9 / 33 timestamp : 1542205196252
--------------------------------------------------------------------------------------------
Mensajes:----> 152
En MD incoming ****************
DODic18 -----> 38.86 / 39.0  size-->  5 / 33 timestamp : 1542205200047
--------------------------------------------------------------------------------------------
Mensajes:----> 153
En MD incoming ****************
DODic18 -----> 38.875 / 39.0  size-->  4 / 33 timestamp : 1542205201368
--------------------------------------------------------------------------------------------
Mensajes:----> 154
En MD incoming ****************
RFX20Dic18 -----> 38400 / 38490  size-->  15 / 17 timestamp : 1542205204860
--------

...

Mensajes:----> 1199
En MD incoming ****************
RFX20Dic18 -----> 38520 / 38560  size-->  25 / 6 timestamp : 1542207439692
--------------------------------------------------------------------------------------------
Mensajes:----> 1200
En MD incoming ****************
RFX20Dic18 -----> 38494 / 38560  size-->  5 / 6 timestamp : 1542207442754
--------------------------------------------------------------------------------------------
Mensajes:----> 1201
En MD incoming ****************
RFX20Dic18 -----> 38450 / 38560  size-->  3 / 6 timestamp : 1542207444059
--------------------------------------------------------------------------------------------
Mensajes:----> 1202
En MD incoming ****************
RFX20Dic18 -----> 38450 / 38560  size-->  3 / 8 timestamp : 1542207444559
--------------------------------------------------------------------------------------------
Mensajes:----> 1203
En MD incoming ****************
RFX20Dic18 -----> 38459 / 38560  size-->  5 / 8 timestamp : 15422074455

Mensajes:----> 1237
En MD incoming ****************
RFX20Dic18 -----> 38451 / 38544  size-->  5 / 2 timestamp : 1542207485994
--------------------------------------------------------------------------------------------
Mensajes:----> 1238
En MD incoming ****************
RFX20Dic18 -----> 38451 / 38543  size-->  5 / 5 timestamp : 1542207486501
--------------------------------------------------------------------------------------------
Mensajes:----> 1239
En MD incoming ****************
RFX20Dic18 -----> 38450 / 38543  size-->  3 / 5 timestamp : 1542207487577
--------------------------------------------------------------------------------------------
Mensajes:----> 1240
En MD incoming ****************
RFX20Dic18 -----> 38450 / 38543  size-->  3 / 3 timestamp : 1542207488082
--------------------------------------------------------------------------------------------
Mensajes:----> 1241
En MD incoming ****************
RFX20Dic18 -----> 38452 / 38543  size-->  5 / 3 timestamp : 154220748920

Mensajes:----> 1275
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38500  size-->  5 / 5 timestamp : 1542207572955
--------------------------------------------------------------------------------------------
Mensajes:----> 1276
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38500  size-->  5 / 4 timestamp : 1542207574095
--------------------------------------------------------------------------------------------
Mensajes:----> 1277
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38500  size-->  5 / 5 timestamp : 1542207576727
--------------------------------------------------------------------------------------------
Mensajes:----> 1278
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38500  size-->  15 / 5 timestamp : 1542207578086
--------------------------------------------------------------------------------------------
Mensajes:----> 1279
En MD incoming ****************
RFX20Dic18 -----> 38460 / 38525  size-->  15 / 1 timestamp : 1542207582

Mensajes:----> 1312
En MD incoming ****************
RFX20Dic18 -----> 38485 / 38525  size-->  5 / 2 timestamp : 1542207726775
--------------------------------------------------------------------------------------------
Mensajes:----> 1313
En MD incoming ****************
RFX20Dic18 -----> 38486 / 38525  size-->  20 / 2 timestamp : 1542207728595
--------------------------------------------------------------------------------------------
Mensajes:----> 1314
En MD incoming ****************
DODic18 -----> 38.787 / 38.843  size-->  1 / 30 timestamp : 1542207729956
--------------------------------------------------------------------------------------------
Mensajes:----> 1315
En MD incoming ****************
RFX20Dic18 -----> 38486 / 38540  size-->  20 / 6 timestamp : 1542207735741
--------------------------------------------------------------------------------------------
Mensajes:----> 1316
En MD incoming ****************
RFX20Dic18 -----> 38486 / 38540  size-->  21 / 6 timestamp : 154220773

Mensajes:----> 1350
En MD incoming ****************
RFX20Dic18 -----> 38502 / 38550  size-->  5 / 22 timestamp : 1542207830117
--------------------------------------------------------------------------------------------
Mensajes:----> 1351
En MD incoming ****************
RFX20Dic18 -----> 38502 / 38550  size-->  6 / 22 timestamp : 1542207831232
--------------------------------------------------------------------------------------------
Mensajes:----> 1352
En MD incoming ****************
RFX20Dic18 -----> 38502 / 38550  size-->  7 / 22 timestamp : 1542207832232
--------------------------------------------------------------------------------------------
BID Size of  RFX20Dic18 -----------------> 7
Mensajes:----> 1353
En MD incoming ****************
DODic18 -----> 38.78 / 38.8  size-->  15 / 50 timestamp : 1542207836453
--------------------------------------------------------------------------------------------
Mensajes:----> 1354
En MD incoming ****************
DODic18 -----> 38.78 / 38.

In [ ]:
pjs.newSingleOrder('ROFX','RFX20Dic18','38436','10','LIMIT','BUY','DAY','555','True')